In [1]:
import decord
import json
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import numpy as np
from tqdm import tqdm

d:\Anaconda\envs\sklearn-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("D:/allforwork/AI/nnsp/NS-2025-10-data/test_data.json", "r") as f:
    test_data = json.load(f)

In [3]:

# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "D:/allforwork/AI/nnsp/Qwen/VL", 
    torch_dtype=torch.bfloat16,
).eval().to("cuda")

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("D:/allforwork/AI/nnsp/Qwen/VL", use_fast=True)

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 19.49it/s]


In [ ]:
res = []

for q in tqdm(test_data[:1]):
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "video",
                    "video": f"D:/allforwork/AI/nnsp/NS-2025-10-data/video/{q['video']}",
                    "max_pixels": 480*480,
                    "fps": 2.0
                },
                {"type": "text", "text": f"Question:{q['question']}?\nPick the best answer in the following choices:\n0.{q['a0']}\n1.{q['a1']}\n2.{q['a2']}\n3.{q['a4']}\n4.{['a4']}\nYour answer is (0/1/2/3/4):"},
            ],
        }
    ]

    # Preparation for inference
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=5)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    res.append(output_text)

  0%|          | 0/1 [00:00<?, ?it/s]qwen-vl-utils using decord to read video.


In [7]:
res

[['1']]